In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['OMP_NUM_THREADS'] = '4'

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using TensorFlow backend.


In [ ]:
os.listdir()

In [2]:
EMBEDDING_FILE = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

train = pd.read_csv('../input/jigsaw/train.csv')
test = pd.read_csv('../input/jigsaw/test.csv')
submission = pd.read_csv('../input/jigsaw/sample_submission.csv')



In [3]:
x=train.iloc[:,2:].sum()
#marking comments without any tags as "clean"
rowsums=train.iloc[:,2:].sum(axis=1)

train['clean']=(rowsums==0)

#count number of clean entries
train['clean'].sum()
print("Total comments = ",len(train))
print("Total clean comments = ",train['clean'].sum())
print("Total tags =",x.sum())

Total comments =  159571
Total clean comments =  143346
Total tags = 35098


In [4]:
train['clean'] = train['clean']*1

In [ ]:
train.head()


In [ ]:
train.clean.value_counts()

In [5]:
val = train['toxic'] + train['severe_toxic'] + train['obscene'] + train['threat'] + train['identity_hate']
new_df = pd.DataFrame(list(zip(train['id'], train['comment_text'], val)), columns= ['id', 'comment_text', 'toxicity'])
new_df

,id,comment_text,toxicity
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [6]:
for i in range(0,len(train)):
    if new_df['toxicity'][i]>0:
        new_df['toxicity'][i]=1
    else:
        new_df['toxicity'][i]=0


In [ ]:
new_df.toxicity.value_counts()

In [8]:
X_train = new_df["comment_text"].values
y_train = new_df['toxicity'].values



max_features = 20000
maxlen = 100
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)

x_train = sequence.pad_sequences(X_train, maxlen=maxlen)



In [15]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val=train_test_split(x_train,y_train, test_size=0.3,random_state=233)


In [16]:
print("Number transactions X_train dataset: ", X_tra.shape)
print("Number transactions y_train dataset: ", y_tra.shape)
#print("Number transactions X_test dataset: ", xx_test.shape)
#print("Number transactions y_test dataset: ", yy_test.shape)

Number transactions X_train dataset:  (111699, 100)
Number transactions y_train dataset:  (111699,)


In [17]:
print("Before OverSampling, counts of label '0'i.e .Clean: {}".format(sum(y_tra==0)))
print("Before OverSampling, counts of label '1' i.e. Toxic: {} \n".format(sum(y_tra==1)))


Before OverSampling, counts of label '0'i.e .Clean: 100472
Before OverSampling, counts of label '1' i.e. Toxic: 11227 



In [18]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_tra, y_tra)

In [19]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '0' i.e Clean: {}".format(sum(y_train_res==0)))
print("After OverSampling, counts of label '1' i.e Toxic: {}".format(sum(y_train_res==1)))


After OverSampling, the shape of train_X: (200944, 100)
After OverSampling, the shape of train_y: (200944,) 

After OverSampling, counts of label '0' i.e Clean: 100472
After OverSampling, counts of label '1' i.e Toxic: 100472


In [20]:
maxlen = 100
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier

In [21]:
embed_size = 128
x = Embedding(max_features, embed_size)(inp)

In [22]:
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)

In [23]:
x = GlobalMaxPool1D()(x)

In [24]:
x = Dropout(0.1)(x)

In [25]:
x = Dense(50, activation="relu")(x)

In [26]:
x = Dropout(0.1)(x)

In [27]:
x = Dense(1, activation="sigmoid")(x)

In [ ]:
X_train_res=np.array(X_train_res)

In [28]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [29]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [30]:
batch_size = 32
epochs = 2
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

model.fit(X_tra,y_tra, batch_size=batch_size, epochs=epochs,validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=2)

Train on 111699 samples, validate on 47872 samples
Epoch 1/2
 - 278s - loss: 0.1392 - accuracy: 0.9519 - val_loss: 0.1041 - val_accuracy: 0.9626

 ROC-AUC - epoch: 1 - score: 0.973075 

Epoch 2/2
 - 267s - loss: 0.0883 - accuracy: 0.9676 - val_loss: 0.1036 - val_accuracy: 0.9632

 ROC-AUC - epoch: 2 - score: 0.972975 



In [32]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          2560000   
_________________________________________________________________
lstm_layer (LSTM)            (None, 100, 60)           45360     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0   